In [7]:
import heapq

def octile_distance(node, goal):
    """
    Octile heuristic for two points (x1, y1) and (x2, y2).
    h = max(dx, dy) + 0.4 * min(dx, dy)
    """
    x1, y1 = node
    x2, y2 = goal
    dx = abs(x2 - x1)
    dy = abs(y2 - y1)
    return max(dx, dy) + 0.4 * min(dx, dy)

def get_neighbors(node):
    """
    Return all valid 8-direction neighbors of 'node' within the 3x3 grid
    plus the movement cost for each neighbor.
    """
    x, y = node
    neighbors = []
    # Possible 8 directions: dx in [-1, 0, 1], dy in [-1, 0, 1], excluding (dx=0, dy=0)
    for dx in [-1, 0, 1]:
        for dy in [-1, 0, 1]:
            if dx == 0 and dy == 0:
                continue
            nx, ny = x + dx, y + dy
            if 0 <= nx <= 2 and 0 <= ny <= 2:  # inside the grid
                cost = move_cost(node, (nx, ny))
                neighbors.append(((nx, ny), cost))
    return neighbors

def move_cost(a, b):
    """
    Return the cost of moving from 'a' to 'b', given the problem's rules:
      - 1.0 for horizontal/vertical
      - 1.4 for diagonal
      - 0.2 for special conveyor belts
    """
    (x1, y1), (x2, y2) = a, b
    dx = abs(x2 - x1)
    dy = abs(y2 - y1)
    
    # Check for conveyor belts:
    # (1,0) <-> (1,1) or (1,2) <-> (2,2)
    conveyor_pairs = {frozenset([(1,0), (1,1)]),
                      frozenset([(1,2), (2,2)])}
    if frozenset([a,b]) in conveyor_pairs:
        return 0.2
    
    # Horizontal or vertical
    if dx + dy == 1:
        return 1.0
    # Diagonal
    else:
        return 1.4

def reconstruct_path(parents, goal):
    """
    Reconstruct the path from the goal back to the start
    using the 'parents' dictionary.
    """
    path = []
    current = goal
    while current is not None:
        path.append(current)
        current = parents[current]
    path.reverse()
    return path

def astar_search(start, goal):
    """
    Perform A* search on the 3x3 grid from 'start' to 'goal'.
    Returns:
      - path: list of (x, y) states from start to goal
      - cost: total path cost
      - visited_order: the order in which nodes were popped from the queue
    """
    # Min-heap priority queue storing (f, g, node)
    # f = g + h, where g = cost so far, h = octile_distance
    pq = []
    # Dictionary to store cost so far and parent
    g_values = {start: 0.0}
    parents = {start: None}
    
    # Push the start
    start_f = g_values[start] + octile_distance(start, goal)
    heapq.heappush(pq, (start_f, g_values[start], start))
    
    visited_order = []  # Track the order in which we pop from pq
    
    while pq:
        f_current, g_current, current = heapq.heappop(pq)
        print("Current:", current)
        print('pq', pq)
        print('visited_order', visited_order)
        print(parents)
        print("======================================")

        visited_order.append(current)
        
        # If this is the goal, reconstruct path and return
        if current == goal:
            path = reconstruct_path(parents, goal)
            return path, g_values[goal], visited_order
        
        # If our f-value is out of date (there is a smaller g-value), skip
        if g_values[current] < g_current:
            continue
        
        # Expand neighbors
        for neighbor, step_cost in get_neighbors(current):
            tentative_g = g_values[current] + step_cost
            
            # If we found a cheaper route to 'neighbor', update
            if neighbor not in g_values or tentative_g < g_values[neighbor]:
                g_values[neighbor] = tentative_g
                parents[neighbor] = current
                f_neighbor = tentative_g + octile_distance(neighbor, goal)
                heapq.heappush(pq, (f_neighbor, tentative_g, neighbor))

    # If we exhaust the queue without finding goal, return failure
    return None, float('inf'), visited_order

if __name__ == "__main__":
    start = (0,0)
    goal = (2,2)
    path, cost, visited_order = astar_search(start, goal)
    print("Path:", path)
    print("Total Cost:", cost)
    print("Visited Order:", visited_order)


Current: (0, 0)
pq []
visited_order []
{(0, 0): None}
Current: (1, 1)
pq [(3.4, 1.0, (0, 1)), (3.4, 1.0, (1, 0))]
visited_order [(0, 0)]
{(0, 0): None, (0, 1): (0, 0), (1, 0): (0, 0), (1, 1): (0, 0)}
Current: (2, 2)
pq [(3.4, 1.0, (0, 1)), (3.4, 1.0, (1, 0)), (3.4, 2.4, (2, 1)), (3.4, 2.4, (1, 2)), (4.8, 2.8, (2, 0)), (4.8, 2.8, (0, 2))]
visited_order [(0, 0), (1, 1)]
{(0, 0): None, (0, 1): (0, 0), (1, 0): (0, 0), (1, 1): (0, 0), (0, 2): (1, 1), (1, 2): (1, 1), (2, 0): (1, 1), (2, 1): (1, 1), (2, 2): (1, 1)}
Path: [(0, 0), (1, 1), (2, 2)]
Total Cost: 2.8
Visited Order: [(0, 0), (1, 1), (2, 2)]
